In [3]:
import json
import numpy as np
import random
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D


In [4]:
# Load the dataset
with open('./dataset chatbot.json') as file:
    data = json.load(file)

In [5]:
# Extract data
patterns = []
tags = []
responses = {}
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

In [6]:
# Tokenize the patterns
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, padding='post')


In [7]:
# Encode the tags
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(tags)


In [8]:
# Build the model
model = Sequential([
    Embedding(input_dim=2000, output_dim=16, input_length=len(padded_sequences[0])),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(set(labels)), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
# Train the model
model.fit(padded_sequences, np.array(labels), epochs=500)

Epoch 1/500
8/8 [==============================] - 1s 4ms/step - loss: 4.3807 - accuracy: 0.0216
Epoch 2/500
8/8 [==============================] - 0s 3ms/step - loss: 4.3756 - accuracy: 0.0388
Epoch 3/500
8/8 [==============================] - 0s 4ms/step - loss: 4.3705 - accuracy: 0.0517
Epoch 4/500
8/8 [==============================] - 0s 4ms/step - loss: 4.3642 - accuracy: 0.0517
Epoch 5/500
8/8 [==============================] - 0s 3ms/step - loss: 4.3572 - accuracy: 0.0517
Epoch 6/500
8/8 [==============================] - 0s 4ms/step - loss: 4.3477 - accuracy: 0.0517
Epoch 7/500
8/8 [==============================] - 0s 3ms/step - loss: 4.3368 - accuracy: 0.0517
Epoch 8/500
8/8 [==============================] - 0s 3ms/step - loss: 4.3227 - accuracy: 0.0517
Epoch 9/500
8/8 [==============================] - 0s 4ms/step - loss: 4.3049 - accuracy: 0.0517
Epoch 10/500
8/8 [==============================] - 0s 4ms/step - loss: 4.2818 - accuracy: 0.0517
Epoch 11/500
8/8 [===========

In [10]:
import pickle
# Save the model and the tokenizer
model.save('chatbot_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Model training complete and saved!")

Model training complete and saved!


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
# Example function to use the model
def chatbot_response(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, padding='post', maxlen=len(padded_sequences[0]))
    pred = model.predict(padded)
    tag = label_encoder.inverse_transform([np.argmax(pred)])
    return random.choice(responses[tag[0]])


In [12]:
# Test the chatbot
print(chatbot_response("What treatment options are available?"))


1/1 [==============================] - 0s 98ms/step
Just as there are different types of medications for physical illness, different treatment options are available for individuals with mental illness. Treatment works differently for different people. It is important to find what works best for you or your child.


In [13]:
import joblib
# Save the trained model
joblib.dump(model, 'chatbotmodel.pkl')

['chatbotmodel.pkl']